In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime
import trimesh 
import time
import math
from vedo import *
from trimesh import creation, transformations
from ipyvtklink.viewer import ViewInteractiveWidget

# Limit GPU memory 
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        memlim = 2*1024
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
    except RuntimeError as e:
        print(e)

# ----------------------------------

print(tf.__version__)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8) # Set default size of plots 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%matplotlib notebook

# For auto loading external modules 
%load_ext autoreload
%autoreload 2
%autosave 180
        

2024-06-10 16:17:02.664884: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 16:17:02.666144: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 16:17:02.693494: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 16:17:02.694098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 16:17:03.152163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

[]
2.13.0


Autosaving every 180 seconds


In [2]:
# Load mesh of object 
surf_fn = "/home/danielchoate/ASAR/Year1/ErrorVectorFields/TestEnvironment.STL"
print(surf_fn)
M = trimesh.load(surf_fn)
# M.rotate[90,0,0]
# Get ground truth translation
# -- import a file to represent the ground truth translation as an txt --
# gt = ...
# print(gt)
# load point clouds of surface generated in matlab 
scale_factor = 1000
trans_base = [1, 1, 0]
trans_base = [i * scale_factor for i in trans_base]
trans_match = [-12.5, -12.5, 2.5]
trans_match = [i * scale_factor for i in trans_match]
print(trans_base)
pts1_fn = "/home/danielchoate/ASAR/Year1/ErrorVectorFields/ptcloud1PTS.txt" 
pts1 = np.loadtxt(pts1_fn)
# print(pts1)
pts1 = pts1*np.array([scale_factor, scale_factor, scale_factor]) + np.array(trans_match)
# print(pts1)
pts2_fn = "/home/danielchoate/ASAR/Year1/ErrorVectorFields/ptcloud2PTS.txt"
pts2 = np.loadtxt(pts2_fn)
print(pts2)
pts2 = pts2*np.array([scale_factor, scale_factor, scale_factor]) - np.array(trans_base) + np.array(trans_match)
print(pts2)


# temp for making wall figure (not used for my example, but keep to see the function)
# pts2 = pts2[pts2[:,1] < -39]
# pts1 = pts1[pts1[:,1] < -40]
# print(np.shape(pts2))

/home/danielchoate/ASAR/Year1/ErrorVectorFields/TestEnvironment.STL
[1000, 1000, 0]
[[-3.7126305e+00 -4.5466610e-16 -1.5000001e+00]
 [-3.7895457e+00 -4.6408550e-16 -1.4999999e+00]
 [-3.8692588e+00 -4.7384754e-16 -1.4999999e+00]
 ...
 [           nan            nan            nan]
 [           nan            nan            nan]
 [           nan            nan            nan]]
[[-17212.6305 -13500.        999.9999]
 [-17289.5457 -13500.       1000.0001]
 [-17369.2588 -13500.       1000.0001]
 ...
 [        nan         nan         nan]
 [        nan         nan         nan]
 [        nan         nan         nan]]


In [3]:
# Rotate point cloud to match plotted mesh

angle = math.pi/2
direction = [1, 0, 0]
center = [0, 0, 0]

rot_matrix = transformations.rotation_matrix(angle, direction, center)

M.apply_transform(rot_matrix)

angle = math.pi/2*3
direction = [0, 0, 1]
center = [0, 0, 0]

rot_matrix = transformations.rotation_matrix(angle, direction, center)

M.apply_transform(rot_matrix)

<trimesh.Trimesh(vertices.shape=(109, 3), faces.shape=(214, 3), name=`TestEnvironment.STL`)>

In [4]:
# plot with vedo 
plt1 = Plotter(N = 1, axes = 4, bg = (1,1,1), interactive = True)
disp = []
disp.append(Points(pts1, c = '#0072BD', r = 3, alpha = 1)) #+ np.array([-13,-13,3]) #0343DF
# disp.append(Points(pts2, c = 'red', r = 3, alpha = 1)) #+ np.array([-13,-13,3])
w1 = Mesh(M).c("white").alpha(1)
# w1.pose(gt[0], gt[1], gt[2]) # ADD WHEN YOU HAVE GROUND TRUTH 
# w1.pos(0,0,0) # 
# print(gt[2]) # ADD WHEN YOU HAVE GROUND TRUTH
disp.append(w1)

print(disp)

plt1.show(disp, "Fig 1.1")
ViewInteractiveWidget(plt1.window)

[<Points(0x61026a0) at 0x7f5b4846b220>, <Mesh(0x610c1a0) at 0x7f5b4846b4c0>]


ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)